## extracting from recon3d

In [1]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv


MASSpy version: 0.1.6


In [2]:
#load Recon3D
model_dir = os.path.abspath("models")
data_dir = os.path.abspath("data")
# model_dir
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
core= load_json_cobra_model(filename=os.path.join(model_dir,"CoreModel.json"))



Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-02


In [3]:
# def metabolite_to_dict(metabolite):
#     new_met = dict()
#     for key in _REQUIRED_METABOLITE_ATTRIBUTES:
#         new_met[key] = _fix_type(getattr(metabolite, key))
#     return new_met
from cobra_dict import metabolite_to_dict
from cobra_dict import reaction_to_dict 

In [4]:
reaction_list=[]
# metabolite_list=[]
for rid in core.reactions:
    reaction_list.append(rid.id)
#     rxn = R3D.reactions.get_by_id(rid)
#     for met in rxn.metabolites:
#         if met.id not in metabolite_list:
#             metabolite_list.append(met.id)
# len(metabolite_list)
# len(reaction_list)

In [5]:
# met_df=pd.DataFrame()
# for met in metabolite_list:
#     r3d_met= R3D.metabolites.get_by_id(met)
#     m=metabolite_to_dict(r3d_met)
#     df_2=pd.DataFrame.from_dict(m,orient='index')
#     df_2=df_2.T
#     met_df=met_df.append(df_2)

# met_df=met_df.set_index('id')
# met_df

In [6]:
# met_df=pd.DataFrame()
# for met in metabolite_list:
#     if met in R3D.metabolites:
#         r3d_met= R3D.metabolites.get_by_id(met)
#         m=metabolite_to_dict(r3d_met)

#     else:
#         core_met = core.metabolites.get_by_id(met)
#         m=metabolite_to_dict(core_met)
#     df_2=pd.DataFrame.from_dict(m,orient='index')
#     df_2=df_2.T
#     met_df= pd.concat([met_df, df_2])

# met_df=met_df.set_index('id')
# met_df

In [7]:
# csv_met = os.path.join(data_dir,"big_core_met_df.csv")
# met_df.to_csv(csv_met)

In [8]:
# for i in to_b_added:
# csv_met = os.path.join(data_dir,"big_core_met_df.csv")
# met_df.to_csv(csv_met)
    

In [9]:
reaction_list

['CSm',
 'ACONTm',
 'ICDHxm',
 'AKGDm',
 'SUCOASm',
 'SUCD1m',
 'FUMm',
 'MDHm',
 'NADH2_u10mi',
 'FADH2ETC',
 'CYOR_u10mi',
 'CYOOm2i',
 'ATPS4mi',
 'Htmi',
 'HEX1',
 'PGI',
 'PFK',
 'FBA',
 'TPI',
 'GAPD',
 'PGK',
 'PGM',
 'ENO',
 'PYK',
 'ADK1',
 'PDHm',
 'LDH_L',
 'G6PDH2r',
 'PGL',
 'GND',
 'RPI',
 'RPE',
 'TKT1',
 'TKT2',
 'TALA',
 'PCm',
 'PEPCKm',
 'FBP',
 'G6PPer',
 'G6Pter',
 'GLCter',
 'MALtm',
 'MDH',
 'PEPCK_re',
 'PGMT',
 'GALUi',
 'GLGNS1',
 'GLBRAN',
 'GLPASE1',
 'GLDBRAN',
 'GLPASE2',
 'GGNG',
 'ACACT1m',
 'HMGCOASm',
 'HMGLm',
 'ADCim',
 'BDHm',
 'OCOAT1m',
 'NDPK1',
 'NDPK1m',
 'NDPK2',
 'HCO3Em',
 'ATPtm',
 'AKGMALtm',
 'ASPTA',
 'ASPTAm',
 'CITtam',
 'CITL',
 'ME2',
 'ACS',
 'ARGN',
 'OCBTm',
 'ARGSS',
 'ARGSL',
 'CBPSam',
 'NH4tm',
 'GDHm',
 'GLNS',
 'GLUN',
 'ALATA_L',
 'CYSO',
 '3SALATAi',
 '3SPYRSP',
 'PGCD',
 'PSERT',
 'PSP_L',
 'GHMT2r',
 'ASNS1',
 'GLYtm',
 'MLTHFtm',
 'THFtm',
 'KHK',
 'FBA2',
 'TRIOK',
 'ALCD19y',
 'GLYK',
 '2AMACHYD',
 'SERHL',
 'ALCD2x',

In [10]:
metabolite_list=[]
met_df=pd.DataFrame()
rxn_df=pd.DataFrame()
reactions_not_in_r3d=[]
for rxn in reaction_list:
    if rxn in R3D.reactions:
        if rxn == "DGAT":
            reaction = core.reactions.get_by_id(rxn)
            r=reaction_to_dict(reaction)
            reactions_not_in_r3d.append(rxn)
            print("ahaha gochu")
        else:
            reaction= R3D.reactions.get_by_id(rxn)
            r=reaction_to_dict(reaction)    
    else:
        reaction = core.reactions.get_by_id(rxn)
        r=reaction_to_dict(reaction)
        reactions_not_in_r3d.append(rxn)

    df=pd.DataFrame.from_dict(r,orient='index')
    df=df.T
    rxn_df= pd.concat([rxn_df, df])
    for met in reaction.metabolites:
        if met.id not in metabolite_list:
            if met.id in R3D.metabolites:
                metabolite= R3D.metabolites.get_by_id(met.id)
                m=metabolite_to_dict(metabolite)

            else:
                metabolite = core.metabolites.get_by_id(met.id)
                m=metabolite_to_dict(metabolite)
            # print(met.id)
            metabolite_list.append(met.id)
            df_2=pd.DataFrame.from_dict(m,orient='index')
            df_2=df_2.T
            met_df= pd.concat([met_df, df_2])


rxn_df=rxn_df.set_index('id')
met_df=met_df.set_index('id')
rxn_df
reactions_not_in_r3d

ahaha gochu


['Htm',
 'H2Otr',
 'PItm',
 'PItr',
 'PEPtm',
 'FACOAL80PROD',
 'GAPT',
 'AGPTAT',
 'LPIN',
 'DGAT',
 'MCOATA',
 'ACOATA',
 'KAS14',
 '30AR40',
 '3HAD40',
 'EAR40y',
 '30AS60',
 '30AR60',
 '3HAD60',
 'EAR60y',
 '30AS80',
 '30AR80',
 '3HAD80',
 'EAR80y',
 'FA80ACPHi',
 'ACOAD3fm',
 'ECOAH3m',
 'HACD3m',
 'ACACT3rm',
 'ACOAD2fm',
 'ECOAH2m',
 'HACD2m',
 'ACACT2rm',
 'ACACT1rm',
 'EX_h_c',
 'EX_pyr_c',
 'EX_h2o_c',
 'EX_pi_c',
 'EX_glc__D_c',
 'EX_lac__L_c',
 'EX_co2_c',
 'EX_o2_c',
 'EX_octa_prod_c',
 'EX_octa_cons_c',
 'EX_urea_c',
 'EX_gln__L_c',
 'EX_acetone_c',
 'EX_bhb_c',
 'EX_glu__L_c',
 'EX_ser__L_c',
 'EX_cys__L_c',
 'EX_gly_c',
 'EX_glygn2_c',
 'EX_Tyr_ggn_c',
 'EX_ala__L_c',
 'EX_nh4_c',
 'EX_so3_c',
 'EX_etoh_c',
 'EX_glyc_3octa_c',
 'EX_fru_c']

In [15]:
len(reactions_not_in_r3d)

60

In [11]:
met_df.loc["octa_c"]

name           Octanoate (n-C8:0)
charge                         -1
formula                   C8H15O2
compartment                     c
Name: octa_c, dtype: object

In [12]:
csv_met = os.path.join(data_dir,"big_core_met_df.csv")
met_df.to_csv(csv_met)

In [13]:
csv_rxn = os.path.join(data_dir,"big_core_reaction_df.csv")
rxn_df.to_csv(csv_rxn)

In [14]:
json_rxn = os.path.join(data_dir,"big_core_reaction_df.json")
rxn_df.to_json(json_rxn
,orient="index"
)   